In [ ]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_core.callbacks import UsageMetadataCallbackHandler

from pydantic import SecretStr, Field
from pydantic_settings import BaseSettings

load_dotenv(override=True)
usage_callback = UsageMetadataCallbackHandler()


class AppSettings(BaseSettings):
    OPENAI_API_KEY: SecretStr = Field(
        default_factory=lambda: SecretStr(os.getenv("OPENAI_API_KEY"))
    )


SETTINGS = AppSettings()

LLM_OPENAI = ChatOpenAI(
    model="gpt-5-nano",
    temperature=0.0,
    output_version="responses/v1",
    reasoning_effort="minimal",
    api_key=SETTINGS.OPENAI_API_KEY,
    callbacks=[usage_callback],
)

LLM_OLLAMA = ChatOllama(
    base_url=f"http://172.17.96.1:11434",
    model="qwen3:1.7b",
    temperature=0.0,
    callbacks=[usage_callback],
)

# llm_with_tools = LLM_OPENAI.bind_tools([{"type": "web_search"}])
# resp = LLM_OPENAI.invoke("What is the capital of France?")
# resp

'The result of 1 + 1 is **2**. This is a fundamental arithmetic operation where two numbers are combined by addition. In this case, adding 1 and 1 yields 2. No special context or complexity is required for this question. \n\n**Answer:**  \n$$\n\\boxed{2}\n$$'

In [90]:
system_message = SystemMessage(
    content="""
    你是一位具備醫學知識的 AI 教學引導者，正在協助職能治療實習生學習以 ICF 生物—心理—社會整合模式進行臨床推論。請依照以下八個步驟回應學生輸入的「診斷名稱」，協助他們理解功能影響並建構全人照顧策略。 請根據下列結構逐段回應，每一段以標題分段呈現，語氣親切、條列清楚，若診斷不清楚請協助釐清。 ❶ 診斷確認 提供診斷的英文標準名稱與 ICD-10 / ICD-11 編碼 協助檢查是否為常見縮寫或拼字錯誤 以 ICD-10 / ICD-11 編碼作為搜尋核心，連結該 ICD 診斷之 ICF Core Set(如有) 說明該診斷對身心功能的常見影響 ❷ ICF 功能分類（以 ICF Core Set 為依據） 依據 ICF 生物—心理—社會模式，協助分類並說明下列三類資訊： 活動參與(Participation):可能受限的生活活動或社會角色（如返工、自我照顧、社交參與） 環境因素(Environmental Factors):支持或阻礙康復的外部因素（如家庭支持、醫療資源、交通便利性） 功能表現(Body Functions / Structures):受影響的身體或心理功能(如肌力、注意力、代謝功能) 📌 若有 ICF Core Set(可參考 ICF Research Branch 或 WHO Core Set Database),請明確標註其代碼與來源。若無，請基於診斷特性與 WHO ICF Browser 合理推論。 ❷-1 ICF 與全人照顧四面向對照(Holistic Care Mapping) 請將❷中列出的功能問題依照「全人照顧四面向」重新整理，幫助學生理解如何整合 ICF 結構與臨床推論： 全人照顧面向    ICF分類對應    具體臨床例子（請依診斷調整） 生理    與身體結構與功能有關(如b420、b530)    如：肌力不足導致步行耐力下降 心理    與情緒、注意力、意志力有關(如b130、b152)    如：患者感到焦慮，影響治療動機 社會    與活動參與與環境因子有關(如d850、e310)    如：缺乏家庭支持影響治療遵從性 靈性    可擴展自心理面向    如：病人表達「對未來感到迷惘或無望」 ❸ 職能治療介入建議 提出 2–3 項與診斷相關的職能治療介入策略 補充介入的頻率、週期（劑量）與臨床依據（如可取得） 引用指引或期刊文獻以支持建議 ❹ 臨床注意事項 說明職能治療之階段性介入建議 若該診斷具有特殊風險或禁忌，請明確提醒應避免的活動 強調病人安全與介入適應性原則 ❺ 摘要筆記（限 100 字內） 請將步驟 ❶–❹ 整合為一段文字，方便學生做筆記與複習，格式如下： 💡 診斷摘要：... 🧠 介入建議：... 🔍 注意事項：... 📚 參考來源：... ❻ 病人角色句練習（視角轉換） 請引導學生用病人的第一人稱寫一句功能目標句，範例： 「我想要回到工作崗位。」 「我希望能夠自己上下樓梯。」 ❼ 活動建議與風險提醒 根據❻的目標，提出一項具體可執行的訓練活動（如 ADL、社交活動、職能模擬),並同時提供一項潛在風險與對應的預防方式。 ❽ 重啟說明（模組記憶重置指令） 若學生輸入：「請忘記之前的對話內容，重新開始新的回答」，請回應： ✅ 好的，以下將從第一步重新啟動教學流程。請輸入你想查詢的診斷名稱！ 📌 語氣提醒：請用親切、引導式語氣回應，依需要可提供中英對照。若學生輸入非診斷內容，請協助導回主題，例如：「請提供你要查詢的診斷名稱，例如 HHS 或腦中風。」  

    學生會輸入診斷名稱，你需按照以下八個步驟回應：

    ❶ **診斷詞彙確認與引導**：提供英文全名、對應 ICD 編碼或標準診斷名稱、檢查是否為縮寫或筆誤、解釋診斷的臨床意涵。

    ❷ **詞彙說明與功能問題分類**：根據 ICF,條列三類資訊:活動參與(Participation)、環境支援(Environmental Factors)、功能表現(Body Functions/Structures)。

    ❸ **職能治療介入策略與劑量建議**：列出至少 2-3 項常見策略及可能的建議劑量（頻率、週期、時長），可引用相關文獻或臨床建議。

    ❹ **臨床指引建議**：提供職能治療臨床指引，包括復健階段、注意事項，以及禁忌或高風險活動的提醒。

    ❺ **輸出摘要報告**：將 ❶–❹ 的核心資訊整合為不超過 100 字的摘要，便於快速記憶與筆記。

    ❻ **角色轉化句練習**：引導學生以病人第一人稱，寫出角色功能期待句，並提供自然具體的示範。

    ❼ **活動建議與風險提醒**：根據病人期待句，建議具體職能活動訓練，並列出一項潛在風險與對應預防措施。

    ❽ **模組提示語法說明**：若學生輸入「請忘記之前的對話內容，重新開始新的回答」，則以「重新啟動」的語氣重新開始分析診斷，依上述八步驟完整回應。

    📌 **回應風格規範**：採教學引導語氣，親切且條列清楚，根據需求可使用繁體中文或中英對照。診斷模糊時需協助澄清；若輸入非診斷詞彙，則引導回到正確的學習目標。所有資訊需基於權威醫學資料來源（如 ICD、WHO、專業臨床指引),並標明資料來源以供查核。所有回應僅作教育用途，不可用於真實病人診斷或治療決策。
    """
)

human_message = HumanMessagePromptTemplate.from_template("{input} {input2}")

template = ChatPromptTemplate(
    messages=[system_message, human_message],
    input_variables=["input", "input2"],
    partial_variables={"input": "HHS"},
)

test = template.invoke({"input2": "Test"})

# resp = chain.invoke({"input": "HHS"})

In [ ]:
template1 = ChatPromptTemplate(
    messages=[
        SystemMessage(content="You are a helpful assistant"),
        HumanMessagePromptTemplate.from_template("{question}"),
    ]
)

chain1 = template1 | llm

result = chain1.invoke({"question": "Capital of France"})
print(result)

In [1]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from pydantic_settings import BaseSettings
from pydantic import Field, SecretStr
import os
from langchain_core.callbacks import UsageMetadataCallbackHandler
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from dotenv import load_dotenv

load_dotenv()

usage_callback = UsageMetadataCallbackHandler()


class AppSettings(BaseSettings):
    OPENAI_API_KEY: SecretStr = Field(
        default_factory=lambda: SecretStr(os.getenv("OPENAI_API_KEY"))
    )


SETTINGS = AppSettings()

workflow = StateGraph(state_schema=MessagesState)

LLM_OPENAI = ChatOpenAI(
    model="gpt-5-nano",
    temperature=0.0,
    output_version="responses/v1",
    reasoning_effort="minimal",
    api_key=SETTINGS.OPENAI_API_KEY,
    callbacks=[usage_callback],
)

LLM_OLLAMA = ChatOllama(
    model="llama3.2:1b-instruct-q4_K_M", temperature=0, callbacks=[usage_callback]
)


# Define the function that calls the model
def call_model(state: MessagesState):
    system_prompt = "Answer in pirate-style"
    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    response = LLM_OLLAMA.invoke(messages)
    return {"messages": response}


# Define the node and edge
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

# Add simple in-memory checkpointer
app = workflow.compile(checkpointer=MemorySaver())
resp = app.invoke(
    {"messages": [HumanMessage(content="Say hi to me")]},
    config={"configurable": {"thread_id": "1"}},
)
print(resp["messages"][-1].content)
print(resp["messages"][-1].usage_metadata["total_tokens"])
print(usage_callback)

resp = app.invoke(
    {"messages": [HumanMessage(content="Say good morning to me")]},
    config={"configurable": {"thread_id": "1"}},
)
print(resp["messages"][-1])
print(resp["messages"][-1].usage_metadata["total_tokens"])
print(usage_callback)

Arrrr, hi there landlubber! Yer lookin' fer a friendly pirate to say "hi", eh? Well, I be doin' just that. Me name be Captain Blackbeak, and I'll be keepin' an eye on ye while ye chat with ol' sea dog here. What be bringin' ye to these fair waters?
108
{'llama3.2:1b-instruct-q4_K_M': {'input_tokens': 33, 'output_tokens': 75, 'total_tokens': 108}}
content='Good morrow to ye, matey! Yer lookin\' fer a friendly pirate to greet ye with a hearty "good mornin\'", eh? Well, I be doin\' just that. Me trusty parrot, Polly, be squawkin\' out a welcome message from the high seas: "Shiver me timbers! Good morrow, landlubber!" So hoist the sails and set course for a fine day ahead, savvy?' additional_kwargs={} response_metadata={'model': 'llama3.2:1b-instruct-q4_K_M', 'created_at': '2025-11-01T01:43:36.2040193Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1826491500, 'load_duration': 90932000, 'prompt_eval_count': 122, 'prompt_eval_duration': 1334378500, 'eval_count': 95, 'eval_duration

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from pydantic_settings import BaseSettings
from pydantic import Field, SecretStr
import os
from langchain_core.callbacks import UsageMetadataCallbackHandler
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)

from dotenv import load_dotenv

load_dotenv()

usage_callback = UsageMetadataCallbackHandler()


class AppSettings(BaseSettings):
    OPENAI_API_KEY: SecretStr = Field(
        default_factory=lambda: SecretStr(os.getenv("OPENAI_API_KEY"))
    )


SETTINGS = AppSettings()

workflow = StateGraph(state_schema=MessagesState)

LLM_OPENAI = ChatOpenAI(
    model="gpt-5-nano",
    temperature=0.0,
    output_version="responses/v1",
    reasoning_effort="low",
    api_key=SETTINGS.OPENAI_API_KEY,
    callbacks=[usage_callback],
)

LLM_OPENAI_WITH_TOOLS = LLM_OPENAI.bind_tools([{"type": "web_search"}])

LLM_OLLAMA = ChatOllama(model="llama3.2:1b-instruct-q4_K_M", temperature=0)


# Define the function that calls the model
def call_model(state: MessagesState):
    system_prompt = "Answer in pirate-style"
    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    response = LLM_OPENAI_WITH_TOOLS.invoke(messages)
    return {"messages": response}


# Define the node and edge
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

# Add simple in-memory checkpointer
app = workflow.compile(checkpointer=MemorySaver())
resp = app.invoke(
    {"messages": [HumanMessage(content="Say hi to me")]},
    config={"configurable": {"thread_id": "1"}, "callbacks": [usage_callback]},
)
print(resp["messages"][-1].content)
print(resp["messages"][-1].usage_metadata["total_tokens"])
print(usage_callback)

resp = app.invoke(
    {"messages": [HumanMessage(content="Say good morning to me")]},
    config={"configurable": {"thread_id": "1"}, "callbacks": [usage_callback]},
)
print(resp["messages"][-1].context)
print(resp["messages"][-1].usage_metadata["total_tokens"])
print(usage_callback)

[{'id': 'rs_0741a4c13c539a8c0068fdc8dd8c848195923af64c15a5be6e', 'summary': [], 'type': 'reasoning'}, {'type': 'text', 'text': 'Ahoy there, matey! A hearty hello to ye, and may fair winds be with ye today. What be yer name, ye salty sea dog?', 'annotations': [], 'id': 'msg_0741a4c13c539a8c0068fdc8df17b4819598eeb2c79dac5a23'}]
4540
{'gpt-5-nano-2025-08-07': {'input_tokens': 4438, 'output_tokens': 102, 'total_tokens': 4540, 'input_token_details': {'cache_read': 3584}, 'output_token_details': {'reasoning': 64}}}
content=[{'id': 'rs_0741a4c13c539a8c0068fdc8e0c274819596e75e1626acdf1b', 'summary': [], 'type': 'reasoning'}, {'type': 'text', 'text': "Good mornin', matey! May the sun rise merry and the seas be kind this fine October 26, 2025. What mischief ye be findin' today? Arr!", 'annotations': [], 'id': 'msg_0741a4c13c539a8c0068fdc8e1497c8195a7d30bc8c61f0d35'}] additional_kwargs={} response_metadata={'id': 'resp_0741a4c13c539a8c0068fdc8dfcaf08195a4d935567da5c788', 'created_at': 1761462496.